## __Speech → Transcript → Call notes__

This Jupyter notebook aims at transforming my meetings' recordings (audio) into calls notes. 

__→ All you need is: an iPhone (Recording), a Mac (to run this Notebook), working API keys.__

The steps are: 

1. get the transcript from the audio using [Whisper](https://openai.com/index/whisper/) (via [GroqCloud](https://console.groq.com/docs/speech-to-text)) 

2. create a note call from the transcript using [OpenAI](http://openai.com/)

In [ ]:
import os, io
from dotenv import load_dotenv
from pydub import AudioSegment
from groq import Groq
from pathlib import Path

load_dotenv()
client = Groq(api_key=os.environ["GROQ_API_KEY"])

print("GROQ API KEY:", repr(os.environ.get("GROQ_API_KEY"))[:10] + "...")

DOWNLOADS = Path.home() / "Downloads"
CHUNK_MS  = 60 * 1000  # We will process audio in 1-minute chunks to avoid API calls errors

for audio_path in DOWNLOADS.glob("*.m4a"):
    print(f"\n→ Processing {audio_path.name}…")
    audio = AudioSegment.from_file(audio_path)
    chunks = [audio[i : i + CHUNK_MS] for i in range(0, len(audio), CHUNK_MS)]
    all_text = []

    for idx, chunk in enumerate(chunks, 1):
        buf = io.BytesIO()
        chunk.export(buf, format="mp3")
        buf.seek(0)

        resp = client.audio.transcriptions.create(
            file=(f"{audio_path.stem}_{idx}.mp3", buf.read()),
            model="whisper-large-v3",
            response_format="verbose_json",
            timeout=120,
        )
        all_text.append(resp.text.strip())

    transcript = "\n\n".join(all_text)
    out_path = audio_path.with_suffix(".txt")
    out_path.write_text(transcript, encoding="utf-8")
    print(f"Saved {out_path}")


In [ ]:
# TRANSCRIPTION -> NOTES
from openai import OpenAI

client = OpenAI(api_key=os.environ["OPENAI_API_KEY"])

for transcript_path in DOWNLOADS.glob("*.txt"):
    print(f"\n→ Processing {transcript_path.name}...")
    
    transcript = transcript_path.read_text(encoding="utf-8")

    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "user", 
            "content": (
                "You are an assistant that converts meeting transcripts into concise, "
                "well-structured call notes. Include key points, decisions, and action items."
            )
            },
            {"role": "user",
            "content": f"Here is the transcript of a meeting:\n\n{transcript}\n\n"
                           "Please summarize it into clear call notes."
            }

        ],
        temperature=0.2,
        max_tokens=800,
    )
    
    summary = response.choices[0].message.content.strip()
    out_path = transcript_path.with_suffix(".summary.txt")
    out_path.write_text(summary, encoding="utf-8")

    print(f"Summared notes saved to {out_path.name}")